In [84]:
def exerciseBoundary(t, T, K, exBoundType, exBoundParams):
    B_t = K
    if exBoundType > 0:
        while exBoundType > 0:
            B_t -= exBoundParams*(np.sqrt(T-t))
            exBoundType -= 1
    if B_t < 0:
        return 0
    else:
        return B_t
    
def MLMCAmOpLevelL(l, N, NperLoop, T, r, sig, S0, K, M, exBoundType, exBoundParams):
    # l given level --> M^l time steps between 0 and T
    # N number of path samples
    # NperLoop number of path samples done in one loop
    # T final time
    # r risk free interest rate
    # sig vol parameter
    # S0 starting underlying value
    # K strike value
    # exBoundType flag which denotes the parametrisation
    # type of the exercise boundary
    # exBoundParams parameters for the exercise boundary
    
    nf = M**l
    nc = M**(l-1)
    
    hf = T/nf
    hc = T/nc
    
    tf = 0
    tc = 0
    sums = np.zeros(4)
    
    # Loop until N paths have been computed in total
    for N1 in range(1, N+1, NperLoop):
        
        # at each step deal with N2 paths
        N2 = min(NperLoop, N-N1+1)
        
        Xf = np.zeros((N2, 4))
        
        # path variables X_t = log(S_t)
        Xf[:,0] = np.log(S0) * np.ones(N2)
        
        # Probability of not having crossed yet
        Xf[:,1] = np.ones(N2)
        
        # Payoff part 1 --> barrier hit during some interval
        Xf[:,2] = np.zeros(N2)
        Xc = Xf
        tf = 0
        tc = 0
        
        if l == 0:
            for n in range(1,nf+1):
                hf = T/nf**2 * (2*(nf-n)+1)
                dWf = np.sqrt(hf)*np.random.randn((N2,1))
                
                Xleft = Xf[:,0]
                Xright = Xleft + (r-sig**2/2)*hf + sig*dWf
                
                leftBarrier = exerciseBoundary(tf, T, K, exBoundType, exBoundParams)
                tf = tf + hf
                tfMid = tf - hf/2
                rightBarrier = exercieseBoundary(tf, T, K, exBoundType, exBoundParams)
                midBarrier = exerciseBoundary(tfMid,T,K,exBoundType,exBoundParams)
                
                if (leftBarrier != 0) and (rightBarrier != 0):
                    Prob1 = np.exp(-2 * max((Xleft - np.log(leftBarrier)).any(), 0) * max((Xright - np.log(rightBarrier)).any(),0) / hf / sig**2)
                    
                else:
                    Prob1 = np.exp(-2 * max((np.exp(Xleft)-leftBarrier).any(), 0) * max((np.exp(Xright)-rightBarrier).any(), 0) / hf / sig**2 / (np.exp(Xleft))**2)
                    
                # update payoff
                Xf[:,2] = Xf[:,2] + Xf[:,1]*Prob1 * max(K-midBarrier,0)*np.exp(-r*tfMid)
                
                # update crossing probability
                Xf[:,1] = Xf[:,1] * (1.0-Prob1)
                # update path variable
                Xf[:,0] = Xright
                
        else:
            for n in range(1,nc):
                dWc = np.zeros((N2,3))
                dt = np.zeros(2)
                
                for m in range(0,2):
                    hf = T/nf**2 * ( 2*(nf-2*(n-1)-m)+1)
                    dt[m] = hf
                    dWf = np.sqrt(hf)*np.random.randn(N2)
                    dWc[:,m] = dWf
                    Xleft = Xf[:,0]
                    
                    # advance path variable
                    Xright = Xleft + (r-sig**2/2)*hf + sig*dWf
                    
                    # evaluate barrier on left end point
                    leftBarrier = exerciseBoundary(tf,T,K,exBoundType,exBoundParams)
                    tf =tf+hf
                    tfMid = tf - hf/2
                    
                    # evaluate barrier on right end point 
                    rightBarrier = exerciseBoundary(tf,T,K,exBoundType,exBoundParams)
                    
                    # evaluate barrier at mid point
                    midBarrier = exerciseBoundary(tfMid,T,K,exBoundType,exBoundParams)
                    
                    # prob of hitting the barrier
                    if (leftBarrier !=0) and (rightBarrier !=0):
                    
                        Prob1=np.exp(-2.0 * max((Xleft-np.log(leftBarrier)).any(),0) * max((Xright-np.log(rightBarrier)).any(),0) /hf/sig**2)
                
                    else:
                        Prob1=np.exp(-2.0 * max((np.exp(Xleft)-leftBarrier).any(),0) * max((np.exp(Xright)-rightBarrier).any(),0) /hf /sig**2/(np.exp(Xleft))**2)
                        
                    
                
                    # update payoff
    
                    Xf[:,2] = Xf[:,2] + Xf[:,1]*Prob1 *max(K-midBarrier,0)*np.exp(-r*(tfMid))
    
                    # update crossing probability
    
                    Xf[:,1] = Xf[:,1]* (1.0-Prob1)
    
                    # update path value
                    Xf[:,0] = Xright
                    
                hc = dt[0] + dt[1]

                Xleft = Xc[:,0]

                # advance path variable

                Xright = Xleft + (r-sig**2/2)*hc + sig*(dWc[:,0]+dWc[:,1])

                lambdas = dt[0]/hc
                Xmid = Xleft + lambdas*(Xright-Xleft) + sig*(dWc[:,0] - lambdas*(dWc[:,0]+dWc[:,1]))

                # evaluate barrier at left end point
                leftBarrier = exerciseBoundary(tc,T,K,exBoundType,exBoundParams)

                tc =tc+hc

                tcMid = tc - hc/2

                # evaluate barrier at right end point
                rightBarrier = exerciseBoundary(tc,T,K,exBoundType,exBoundParams)

                # evaluate barrier at mid point
                midBarrier2 = exerciseBoundary(tcMid,T,K,exBoundType,exBoundParams)

                # prob of hitting the barrier
                if (leftBarrier !=0) and (rightBarrier !=0):
                
                    logLeftBarrier = np.log(leftBarrier)
                    logRightBarrier = np.log(rightBarrier)
                    logMidBarrier1 = logLeftBarrier + lambdas*(logRightBarrier-logLeftBarrier)
                    Prob11 = np.exp(-2*max((Xleft-logLeftBarrier).any(),0) * max((Xmid-logMidBarrier1).any(),0)     /dt[0]/sig**2)
                    Prob12 = np.exp(-2*max((Xmid-logMidBarrier1).any(),0) * max((Xright-logRightBarrier).any(),0) /dt[1]/sig**2)
            
                else:
                
                    midBarrier1 = leftBarrier + lambdas*(rightBarrier-leftBarrier)
                    Prob11 = np.exp(-2*max((np.exp(Xleft)-leftBarrier).any(),0) * max((np.exp(Xmid)-midBarrier1).any(),0) /dt[0]/sig**2)/(np.exp(Xleft))**2
                    Prob12 = np.exp(-2*max((np.exp(Xmid)-midBarrier1).any(),0)* max((np.exp(Xright)-rightBarrier).any(),0)/dt[1]/sig**2/(np.exp(Xleft))**2)
            
                
                Prob1 = (1.0-(1.0-Prob11)*(1.0-Prob12))
                # update payoff
                Xc[:,2] = Xc[:,2] + Xc[:,1]*Prob1*max(K-midBarrier2,0)*np.exp(-r*tcMid)
                # update crossing probability
                Xc[:,1] = Xc[:,1] * (1-Prob1)
                # update path variable
                Xc[:,0] = Xright
        
        Pf = Xf[:,2] + Xf[:,1]*max((K-np.exp(Xf[:,0])).any(),0)*np.exp(-r*T)

        if l == 0:
            Pc = 0
        else:
            Pc = Xc[:,2] + Xc[:,1]*max((K-np.exp(Xc[:,0])).any(),0)*np.exp(-r*T)
    sums[0] += sum(Pf-Pc)
    sums[1] += sum((Pf-Pc)**2)
    sums[2] += sum(Pf)
    sums[3] += sum(Pf**2)
    
    return sums
    
def MLMCAmOp(eps, L, N, NperLoop, T, r, sig, S0, K, exBoundType, exBoundParams):
    converged = 0
    
    while not converged:
        # Initial Variance Estimate
        L = L+1
        sums = MLMCAmOpLevelL(L, N, NperLoop, T, r, sig, S0, K, exBoundType, exBoundParams)
        sum1[0, L] = N
        sum1[1, L] = sums[0]
        sum1[2, L] = sums[1]
        
        # Optimal Sample Sizes
        
        V1 = sum1[2,:]/sum1[0,:] - (sum1[1,:]/sum1[0,:])**2
        N1 = np.ceil(2*np.sqrt(V1/(2**np.arange(0,L))) * sum(np.sqrt(V1*(2**np.arange(0,L)))) / eps**2)
        
        # Update Sample Sums
        for l in range(0,L+1):
            dNl = Nl[l]-suml[1,l]
            if dNl>0:
                sums = MLMCAmOpLevelL(l, dNl, NperLoop, T, r, sig, S0, K, exBoundType, exBoundParams)
                suml[0,l] += dNl
                suml[1,l] += sums[0]
                suml[2,l] += sums[1]
        
        # Test for Convergence
        ranges = np.linspace(-1,0,2)
        if L>1 and 2**L >= 16:
            con = 2**ranges * suml[1,L+ranges] / suml[0,L+ranges]
            converged = (max(abs(con)) < eps/sqrt(2))
            
        # Evaluate Multi-timestep Estimator
        P = sum(sum1[1, 0:L] / sum1[0, 0:L])
        N1 = sum1[0,:]
        
        return [P, N1]

In [ ]:
import numpy as np

M=2
T = 1
r = 0.05
sig = 0.2
S0 = 0.8
NperLoop = 10000
K = 1.0
L = -1
N = 1000

NperLoop=5000
T=1.0
r=0.05
sig=0.2
S0=1.0
K=1.0
N=1000000

L = np.arange(1,9,1)
exBoundType = 3
exBoundParams = 0.8
del1 = []
del2 = []
var1 = []
var2 = []

for l in L:
    sums = MLMCAmOpLevelL(l,N,NperLoop,T,r,sig,S0,K,M,exBoundType,exBoundParams)
    del1 = [del1, sums[2]/N ]
    del2 = [del2, sums[0]/N ]
    var1 = [var1, sums[3]/N-(sums[2]/N)**2 ]
    var2 = [var2, sums[1]/N-(sums[0]/N)**2 ]
    
print(del1(length(L)))
Eps = [ 0.002, 0.001, 0.0005, 0.0002, 0.0001]
maxl = 0
mlmc_cost = []
std_cost = []
Nls = np.zeros(len(Eps), 1)

for i in range(1,len(Eps)):
    eps = Eps[i]
    [P, Nl] = MLMCAmOp(eps, L, N, NperLoop, T, r, sig, S0, K, exBoundType, exBoundParams)
    l = len(Nl)-1
    maxl = max(maxl,l)
    mlmc_cost += sum(Nl*M**np.arrange(0,l))
    std_cost += (2*var1(l+1)/eps**2)*M**l
    Nls[i,1] = Nl

maxl = max(maxl,4)
nvert = 2
plt.subplot(121)
plt.plot(L, np.log(var1)/np.log(M), L[2:],np.log(var2[2:])/log(M))

plt.subplot(122)
plt.plot(L,lnp.og(abs(del1))/log(M), L[2:],np.log(abs(del2[2:]))/log(M))
plt.show()
